### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [3]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0068646,10,1381620027
1,1,0113277,10,1379466669
2,2,0422720,8,1412178746
3,2,0454876,8,1394818630
4,2,0790636,7,1389963947


In [4]:
movies.groupby(by='movie_id').count().shape[0]

32454

In [5]:
movies.movie_id.value_counts().size

32454

In [6]:
reviews.groupby(by='rating').count().shape[0]

11

In [7]:
movies.groupby(by='genre').genre.count().shape[0]

1210

In [8]:
movies.groupby(by='genre').genre.nunique().size

1210

In [9]:
reviews.groupby(by='user_id').count().shape[0]

55528

In [10]:
reviews.groupby(by='user_id').user_id.nunique().size

55528

In [14]:
len(np.unique(reviews.user_id))

55528

In [19]:
reviews.user_id.nunique()

55528

In [20]:
reviews.isnull().sum(axis=0)

user_id      0
movie_id     0
rating       0
timestamp    0
dtype: int64

In [21]:
reviews.describe().loc['min']

rating    0.0
Name: min, dtype: float64

In [22]:
reviews.describe().loc['max']

rating    10.0
Name: max, dtype: float64

In [23]:
reviews.describe().loc['mean']

rating    7.299672
Name: mean, dtype: float64

In [24]:
reviews.rating.mean()

7.2996718911112932

In [17]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': 
'The number of ratings in the dataset':
'The number of different genres':
'The number of unique users in the dataset':
'The number missing ratings in the reviews dataset':
'The average rating given across all ratings':
'The minimum rating given across all ratings':
'The maximum rating given across all ratings':
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [25]:
movies['movie'][0].strip().split('(')[-1]

'1894)'

In [26]:
def date_century_text(year_text):
    yr = year_text[0:2]
    if(yr == '18'):
        return "1800's"
    if(yr == '19'):
        return "1900's"
    else:
        return "2000's"
    

In [27]:
# movies['date'] = movies['movie'].apply(lambda x: x.strip().split('(')[-1].strip(')')[:]).astype('int64')
movies['date'] = movies['movie'].apply(lambda x: x.strip().split('(')[-1].strip(')')[:])
movies['date_century'] = movies['movie'].apply(lambda x: date_century_text(x.strip().split('(')[-1].strip(')')[:]))

In [28]:
movies['genre_array'] =movies['genre'].str.split(pat='|')

In [29]:
movies.head()

,movie_id,movie,genre,date,date_century,genre_array
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1800's,"[Documentary, Short]"
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1800's,"[Documentary, Short]"
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1800's,"[Documentary, Short]"
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1800's,NaN
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1800's,"[Short, Horror]"


In [30]:
movies.genre_array[0]

['Documentary', 'Short']

In [32]:
genre_array_dropna = movies.genre_array.dropna()

In [33]:
#genre_array_dropna

In [34]:
#movies['genre_array'].apply(pd.Series).stack() slow

In [35]:
import math
genre_list = []
for row in genre_array_dropna:
    for genre in row:
        if((genre not in genre_list)):
            genre_list.append(genre)

In [36]:
genre_list

['Documentary',
 'Short',
 'Horror',
 'Comedy',
 'Adventure',
 'Fantasy',
 'Action',
 'Crime',
 'Drama',
 'Romance',
 'History',
 'Family',
 'War',
 'Sport',
 'Biography',
 'Mystery',
 'Sci-Fi',
 'Western',
 'Thriller',
 'Animation',
 'Music',
 'Musical',
 'Film-Noir',
 'Adult',
 'Talk-Show',
 'News',
 'Reality-TV',
 'Game-Show']

In [37]:
len(genre_list)

28

In [38]:
# [0]*28

In [39]:
'Short'in genre_list

True

In [76]:
text = 'Short Documetary test' #val
print(text.find('Short'))
print(text.find('Drama'))

0
-1


In [40]:
def fill_one_hot_code(input_):
    array_one_hot = [0]*len(genre_list)
    if(type(input_)==list):
        
        for item in input_:
            for i, genre in enumerate(genre_list):
                if(item == genre):
                    array_one_hot[i] = 1
        
    return array_one_hot
            

In [41]:
fill_one_hot_code(movies['genre_array'][150])

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [42]:
movies['genre_array_one_hot'] = movies['genre_array'].apply(lambda x:fill_one_hot_code(x))

In [43]:
# movies['genre_array_one_hot'].values.tolist()

In [44]:
movies.head()

,movie_id,movie,genre,date,date_century,genre_array,genre_array_one_hot
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1800's,"[Documentary, Short]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1800's,"[Documentary, Short]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1800's,"[Documentary, Short]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1800's,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1800's,"[Short, Horror]","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [45]:
movies_genre_one_hot = pd.DataFrame(movies['genre_array_one_hot'].values.tolist(), columns=genre_list)

In [46]:
fill_one_hot_code(['Short','Documentary','Western'])

[1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [48]:
movies_date = pd.get_dummies(data=movies[['date_century']], columns=['date_century'])

In [49]:
movies_date.head()

,date_century_1800's,date_century_1900's,date_century_2000's
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [50]:
movies_genre_one_hot.head()

,Documentary,Short,Horror,Comedy,Adventure,Fantasy,Action,Crime,Drama,Romance,...,Thriller,Animation,Music,Musical,Film-Noir,Adult,Talk-Show,News,Reality-TV,Game-Show
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
movies.head()

,movie_id,movie,genre,date,date_century,genre_array,genre_array_one_hot
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1800's,"[Documentary, Short]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1800's,"[Documentary, Short]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1800's,"[Documentary, Short]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1800's,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1800's,"[Short, Horror]","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [52]:
movies_new = pd.concat([movies,movies_date,movies_genre_one_hot],axis=1)

In [53]:
movies_new.head()

,movie_id,movie,genre,date,date_century,genre_array,genre_array_one_hot,date_century_1800's,date_century_1900's,date_century_2000's,...,Thriller,Animation,Music,Musical,Film-Noir,Adult,Talk-Show,News,Reality-TV,Game-Show
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1800's,"[Documentary, Short]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1800's,"[Documentary, Short]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1800's,"[Documentary, Short]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1800's,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1800's,"[Short, Horror]","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
movies_new=movies_new.drop(['genre_array','date_century','genre_array_one_hot'],axis=1)

In [55]:
movies_new.head()

,movie_id,movie,genre,date,date_century_1800's,date_century_1900's,date_century_2000's,Documentary,Short,Horror,...,Thriller,Animation,Music,Musical,Film-Noir,Adult,Talk-Show,News,Reality-TV,Game-Show
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [56]:
from datetime import datetime
reviews['date'] = reviews['timestamp'].apply(lambda x: datetime.utcfromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))

In [57]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0068646,10,1381620027,2013-10-12 23:20:27
1,1,0113277,10,1379466669,2013-09-18 01:11:09
2,2,0422720,8,1412178746,2014-10-01 15:52:26
3,2,0454876,8,1394818630,2014-03-14 17:37:10
4,2,0790636,7,1389963947,2014-01-17 13:05:47


In [58]:
reviews['date'] = pd.to_datetime(reviews['date'])
reviews['year'] = reviews['date'].dt.year
reviews['month'] = reviews['date'].dt.month

In [59]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,year,month
0,1,0068646,10,1381620027,2013-10-12 23:20:27,2013,10
1,1,0113277,10,1379466669,2013-09-18 01:11:09,2013,9
2,2,0422720,8,1412178746,2014-10-01 15:52:26,2014,10
3,2,0454876,8,1394818630,2014-03-14 17:37:10,2014,3
4,2,0790636,7,1389963947,2014-01-17 13:05:47,2014,1


In [60]:
reviews_new = pd.get_dummies(data=reviews[['year', 'month']], columns=['year', 'month'])

In [61]:
reviews_new.head()

,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [62]:
reviews_new = pd.concat([reviews,reviews_new],axis=1)

In [63]:
reviews_new.head()

,user_id,movie_id,rating,timestamp,date,year,month,year_2013,year_2014,year_2015,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,1,0068646,10,1381620027,2013-10-12 23:20:27,2013,10,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,0113277,10,1379466669,2013-09-18 01:11:09,2013,9,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,2,0422720,8,1412178746,2014-10-01 15:52:26,2014,10,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,2,0454876,8,1394818630,2014-03-14 17:37:10,2014,3,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,2,0790636,7,1389963947,2014-01-17 13:05:47,2014,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Another way

In [ ]:
# number of different genres
genres = []
for val in movies.genre:
    try:
        genres.extend(val.split('|'))   # ad to existing list
    except AttributeError:
        pass

# we end up needing this later
genres = set(genres)
print("The number of genres is {}.".format(len(genres)))

In [ ]:
# pull date if it exists
create_date = lambda val: val[-5:-1] if val[-1] == ')' else np.nan

# apply the function to pull the date
movies['date'] = movies['movie'].apply(create_date)

# Return century of movie as a dummy column
def add_movie_year(val):
    if val[:2] == yr:
        return 1
    else:
        return 0
        
# Apply function
for yr in ['18', '19', '20']:
    movies[str(yr) + "00's"] = movies['date'].apply(add_movie_year)

In [ ]:
# Function to split and return values for columns
def split_genres(val):
    try:
        if val.find(gene) >-1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0

# Apply function for each genre
for gene in genres:        
    movies[gene] = movies['genre'].apply(split_genres)

In [74]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1     68646      10  1381620027  2013-10-12 23:20:27   
1           1        1    113277      10  1379466669  2013-09-18 01:11:09   
2           2        2    422720       8  1412178746  2014-10-01 15:52:26   
3           3        2    454876       8  1394818630  2014-03-14 17:37:10   
4           4        2    790636       7  1389963947  2014-01-17 13:05:47   

   month_1  month_2  month_3  month_4    ...      month_9  month_10  month_11  \
0        0        0        0        0    ...            0         1         0   
1        0        0        0        0    ...            0         0         0   
2        0        0        0        0    ...            0         1         0   
3        0        0        0        0    ...            0         0         0   
4        0        0        0        0    ...            0         0         0   

   month_12  year_2013  year_2014  year_2015  year

In [75]:
movies_new.head()

,Unnamed: 0,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [274]:
movies_new.columns

Index(['movie_id', 'movie', 'genre', 'date', 'date_century_1800's',
       'date_century_1900's', 'date_century_2000's', 'Documentary', 'Short',
       'Horror', 'Comedy', 'Adventure', 'Fantasy', 'Action', 'Crime', 'Drama',
       'Romance', 'History', 'Family', 'War', 'Sport', 'Biography', 'Mystery',
       'Sci-Fi', 'Western', 'Thriller', 'Animation', 'Music', 'Musical',
       'Film-Noir', 'Adult', 'Talk-Show', 'News', 'Reality-TV', 'Game-Show'],
      dtype='object')